_THIS `.qmd` FILE SHOULD BE USED TO WRITE YOUR REPORT.  YOU WILL NEED TO COMPILE THE REPORT INTO A `.html` DOCUMENT AND SUBMIT IT ON CANVAS._


In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import altair as alt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import tree

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, RocCurveDisplay

from IPython.display import Markdown
from IPython.display import display
from tabulate import tabulate

## Elevator pitch

_A SHORT (4-5 SENTENCES) PARAGRAPH THAT `SUMMARIZES` THE WORK REQUESTED INCLUDING THE TOP RESULTS._

_A Client has requested this analysis and  this is your one shot of  what you would say to your boss in a 2min elevator ride before he takes your report and hands it to the client._


In [ ]:
#| label: project data
#| code-summary: Read and format project data
# Include and execute your code here
url = 'https://raw.githubusercontent.com/fivethirtyeight/data/master/star-wars-survey/StarWars.csv'
df = pd.read_csv(url, encoding='ISO-8859-1')
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('data_server')

__Highlight the grand questions__

## GRAND QUESTION 1

__Shorten the column names and clean them up for easier use with pandas. Provide a table or list that exemplifies how you fixed the names.__



_Renaming the Columns_

In [ ]:
#| label: GQ1
#| code-summary: Read and format data
# Include and execute your code here
column_df = pd.DataFrame({'header1': df.columns, "header2": df.iloc[0, :]}).reset_index(drop=True)
column_df.header1[1] = "star_wars_watched"
column_df.header1[2] = "star_wars_fan"
column_df.header1[3] = "have_you_seen_"
column_df.header1[9] = "rank_movie_"
column_df.header1[15] = "favorable_character_"
column_df.header1[29] = "shot_first"
column_df.header1[30] = "expanded_universe_familiar"
column_df.header1[31] = "expanded_universe_fan"
column_df.header1[32] = "star_trek_fan"
column_df.header1[35] = "household_income"
column_df.header1[37] = "location"

column_df.header2[3] = "episode_1"
column_df.header2[4] = "episode_2"
column_df.header2[5] = "episode_3"
column_df.header2[6] = "episode_4"
column_df.header2[7] = "episode_5"
column_df.header2[8] = "episode_6"
column_df.header2[9] = "episode_1"
column_df.header2[10] = "episode_2"
column_df.header2[11] = "episode_3"
column_df.header2[12] = "episode_4"
column_df.header2[13] = "episode_5"
column_df.header2[14] = "episode_6"
column_df.header2[15] = "Han_Solo"
column_df.header2[16] = "Luke_Skywalker"
column_df.header2[17] = "Princess_Leia"
column_df.header2[18] = "Anakin_Skywalker"
column_df.header2[19] = "Obi_Wan_Kenobi"
column_df.header2[20] = "Emperor_Palpatine"
column_df.header2[21] = "Darth_Vader"
column_df.header2[22] = "Lando_Calrissian"
column_df.header2[23] = "Boba_Fett"
column_df.header2[24] = "C_3P0"
column_df.header2[25] = "R2_D2"
column_df.header2[26] = "Jar_Jar_Binks"
column_df.header2[27] = "Padme_Amidala"

column_df.replace(["Unnamed:", "Response"], np.nan, regex=True, inplace=True)
column_df.header2.fillna("", inplace=True)

column_df.ffill(inplace=True)

header = column_df.header1 + column_df.header2

df = pd.read_csv(url, encoding='ISO-8859-1', header=None, skiprows=2, names=header)

_Table of Column Names_

In [ ]:
header

## GRAND QUESTION 2

__Clean and format the data so that it can be used in a machine learning model. As you format the data, you should complete each item listed below. In your final report provide example(s) of the reformatted data with a short description of the changes made.__

_Filter the dataset to respondents that have seen at least one film._


In [ ]:
columns_to_convert = ['star_wars_watched', 'star_wars_fan', 'expanded_universe_familiar', 'expanded_universe_fan', 'star_trek_fan']
for col in columns_to_convert:
    df[col] = df[col].replace({'Yes': 1, 'No': 0})

columns_to_mean = ['star_wars_watched', 'star_wars_fan', 'expanded_universe_familiar', 'expanded_universe_fan', 'star_trek_fan']
mean_values = df[columns_to_mean].mean()

df[columns_to_mean] = df[columns_to_mean].fillna(mean_values)

df = df[df['star_wars_watched'] == 1]

episodes_seen = ['have_you_seen_episode_1', 'have_you_seen_episode_2', 'have_you_seen_episode_3', 'have_you_seen_episode_4', 'have_you_seen_episode_5', 'have_you_seen_episode_6']
for col in episodes_seen:
    df[col] = df[col].replace({'^.*Star Wars.*$': 1}, regex=True)
    df[col] = df[col].fillna(0)

df_movies = df[episodes_seen]
df_movies = df_movies.loc[~(df_movies == 0).all(axis=1)]

df_shot_first = df[df['shot_first'].notnull()]


df.head(5)


_Create a new column that converts the age ranges to a single number. Drop the age range categorical column._


In [ ]:
df = df.dropna(subset=['Gender'])
df['Gender'] = df['Gender'].replace({'^.*Male.*$': 1, '^.*Female.*$': 0}, regex=True)

df['Age'] = df['Age'].replace({'^.*18-29.*$': 1, '^.*30-44.*$': 2, '^.*45-60.*$': 3, '^.*> 60.*$': 4}, regex=True)

df['Age'].value_counts(dropna=False)


_Create a new column that converts the education groupings to a single number. Drop the school categorical column_


In [ ]:
df['Education'] = df['Education'].replace({
    '^Bachelor degree$': 1,
    '^High school degree$': 2,
    '^Some college or Associate degree$': 3,
    '^Graduate degree$': 4,
    '^Less than high school degree$': 5,
    np.nan: 6
}, regex=True)

df['Education'].value_counts(dropna=False)

_Create a new column that converts the income ranges to a single number. Drop the income range categorical column._


In [ ]:
df['household_income'].fillna(0, inplace=True)

df['household_income'] = df['household_income'].replace({
    '$0 - $24,999': 1,
    '$25,000 - $49,999': 2,
    '$50,000 - $99,999': 3,
    '$100,000 - $149,999': 4,
    '$150,000+': 5,
})

df['household_income'] = df['household_income'].astype(float)



df['household_income'].value_counts(dropna=False)


_Create your target (also known as “y” or “label”) column based on the new income range column._


In [ ]:
df['target'] = (df['household_income'] >= 3).astype(int)

df['target'].value_counts(dropna=False)


_One-hot encode all remaining categorical columns._


In [ ]:
dummies = pd.get_dummies(df['location'], prefix='Location_')

df = pd.concat([df, dummies], axis=1)


df.drop('location', axis=1, inplace=True)


df.head()


## GRAND QUESTION 3

__Validate that the data provided on GitHub lines up with the article by recreating 2 of the visuals from the article.__

_Chart 1_


In [ ]:
total_count = len(df_shot_first)

df_count = df_shot_first.groupby('shot_first').size().reset_index(name='count')
df_count['shot_first'] = df_count['shot_first'].replace({0: "I don't understand this question", 1: "Han Solo", 2: "Greedo"})
df_count['percentage'] = df_count['count'] / total_count * 100


title = "Who Shot First?"
bars = ['Han Solo', 'Greedo', "I don't understand this question"]

chart = alt.Chart(df_count).mark_bar().encode(
    x=alt.X('percentage:Q', title='Percentage'),
    y=alt.Y('shot_first:N', sort=bars, title=None),
    tooltip=[alt.Tooltip('percentage:Q', format='.1f')],
).properties(
    title=title,
)

chart


_Chart 2_


In [ ]:
df_movies = df[episodes_seen]
df_movies = df_movies.loc[~(df_movies == 0).all(axis=1)]

film_names = {
    1: 'The Phantom Menace',
    2: 'Attack of the Clones',
    3: 'Revenge of the Sith',
    4: 'A New Hope',
    5: 'The Empire Strikes Back',
    6: 'Return of the Jedi'
}

# Calculate the sum of each column
episodes_sum = {col: df_movies[col].sum() for col in episodes_seen}

# Calculate the percentage of respondents who watched each episode
episodes_percentages = {film_names[int(episode[-1])]: count / len(df_movies) * 100 for episode, count in episodes_sum.items()}

# Create the chart
chart_data = pd.DataFrame({'Film': list(episodes_percentages.keys()), 'Percentage': list(episodes_percentages.values())})

chart = alt.Chart(chart_data).mark_bar().encode(
    x=alt.X('Percentage:Q', title='Percentage of Respondents who Watched'),
    y=alt.Y('Film:N', title=None, sort=list(film_names.values())),
    tooltip=[alt.Tooltip('Percentage:Q', format='.2f')]
).properties(title="Which 'Star Wars' Films Have You Seen?")

chart


## GRAND QUESTION 4

__Build a machine learning model that predicts whether a person makes more than $50k. Describe your model and report the accuracy.__

_Model_


In [ ]:
X_pred = df.filter(regex='Education|household_income|Age')
y_pred = df.filter(regex = 'target')
X_train, X_test, y_train, y_test = train_test_split(
X_pred, y_pred, test_size = .34, random_state = 76)  

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_probs = clf.predict_proba(X_test)


In [ ]:
print(metrics.classification_report(y_pred, y_test))

_Accuracy_


In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

y_probs = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_probs)


roc_auc = auc(fpr, tpr)


roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Decision Tree')
roc_display.plot()
plt.show()

In [ ]:
df_features = pd.DataFrame(
    {'f_names': X_train.columns, 
    'f_values': clf.feature_importances_}).sort_values('f_values', ascending = False)

In [ ]:
(alt.Chart(df_features.query('f_values > .011'))
    .encode(
        alt.X('f_values'),
        alt.Y('f_names', sort = '-x'))
    .mark_bar())